# 멜론 장르별 차트 크롤링

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager # 자동으로 크롬드라이버(가상브라우저) 파일을 다운로드해주는 라이브러리
from selenium.webdriver.chrome.service import Service # 다운로드된 크롬드라이버 파일을 연결하기 위해 활용
import time
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def chrome_setting():
    chrome_options = webdriver.ChromeOptions()
#     chrome_options.add_argument('--headless') # 크롬 탭이 나타나지 않게 옵션 조정
    chrome_options.add_argument('--no-sandbox')
    # 공유 메모리를 담당하는 디렉토리 사용 x
    chrome_options.add_argument('--disable-dev-shm-usage')
    # 본인 user-agent값 입력
    chrome_options.add_argument('본인 user-agent')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

```
StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
```
- 중간중간 뜨는 에러 메세지는 웹페이지가 불러와지기 전에 html을 긁으려고 해서 발생하는 문제임
- 곡 정보를 불러오는게 딜레이가 없어서 발생한듯
1. `time.sleep()` 추가 (`driver.implicitly_wait(10)` 을 사용해도 에러 해결이 안됨)
2. 또는 `WebDriverWait()` 시간 추가
<br><br>
 - `driver.implicitly_wait(10)` : 10초안에 웹페이지를 load 하면 바로 넘어가거나, 10초를 기다림
 - `time.sleep(10)` : 10초를 기다림
 
 <br><br>
 - 페이지 하단 인덱스(y)를 돌 때 XPATH 에서 첫번째 페이지 -> 세 번째 페이지로 도는 오류 발견함
   - css selector로 변경해서 사용했더니 해결(첫번째 인덱스 스킵하기 위해 y+1값 사용)

In [ ]:
# 웹드라이버 오픈
driver = chrome_setting()
driver.implicitly_wait(10)

# 장르 - 발라드 - 인기순 url
# 장르, 인기순, 인덱스 넘버 확인
url = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0100#params%5BgnrCode%5D=GN0100&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex=56501"
driver.get(url)

# 빈 리스트 변수 정의
title_list = []
lyrics_list = []
singer_list = []
date_list = []
like_list = []
genre_list = []

# range값 지정
## x : 하단 페이지에서 "다음" 클릭 횟수
## y : 하단 페이지 인덱스 도는 범위 (1~10 페이지 모두 필요하기 때문에 range(1,10) 고정)
## z : 선택된 페이지 내 곡 정보 클릭하는 횟수 (한 페이지에 50곡 있기 때문에 range(1,51) 고정)
for x in range(1,8):
    for y in range(1,10):
        for z in range(1, 51):
            time.sleep(5)
            # element가 로딩될 때 까지 10초 대기
            click_element3 = WebDriverWait(driver, 10) \
                .until(EC.presence_of_element_located((By.CSS_SELECTOR, f'#frm > div > table > tbody > tr:nth-child({z}) > td:nth-child(4) > div > a')))
            click_element3.click()

            try:
                # 제목
                title = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.info > div.song_name")))

                # 가사
                lyrics = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#d_video_summary")))
                
                # 가수
                singer = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.info > div.artist > a > span:nth-child(1)")))
                
                # 발매일
                date = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(4)")))
                
                # 좋아요 수
                like = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#d_like_count")))
                
                # 장르
                genre = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(6)")))
                
                title_list.append(title.text)
                lyrics_list.append(lyrics.text)
                singer_list.append(singer.text)
                date_list.append(date.text)
                like_list.append(like.text)
                genre_list.append(genre.text)

            except:
                print(x,y,z,"에서 에러 발생.")
            
            # 뒤로가기
            driver.back()
            
            
        # 하단 페이지 인덱스 클릭 (두번째 ~ 열번째)
        time.sleep(1)
        click_element2 = WebDriverWait(driver, 2) \
            .until(EC.presence_of_element_located((By.CSS_SELECTOR, f'#pageObjNavgation > div > span > a:nth-child({y+1})')))
        click_element2.click()

    time.sleep(1)

    # 이 코드는 곡 인덱스 501이상부터 크롤링 할 때 ######
    a_tag = 'a[2]'
    ####################################

    # 이 코드는 처음부터 크롤링 할 때 #########
#     a_tag = 'a' if x==1 else 'a[2]'
    ####################################

    # 하단 페이지에서 "다음" 버튼 클릭
    click_element1 = WebDriverWait(driver, 2) \
        .until(EC.presence_of_element_located((By.XPATH, f'//*[@id="pageObjNavgation"]/div/{a_tag}')))
    click_element1.click()

In [ ]:
raw_result = {'제목': title_list,
              '가사': lyrics_list,
              '가수' : singer_list,
              '발매일' : date_list,
              '좋아요수' : like_list,
              '장르': genre_list}

result = pd.DataFrame(raw_result)

In [ ]:
print(result.shape)
result.tail(10)

In [ ]:
# csv 파일로 save
# result.to_csv("data/melon_ballad_56501_60000.csv", index=False)